
<a id='whatsnew-0180'></a>

# v0.18.0 (March 13, 2016)

{{ header }}

This is a major release from 0.17.1 and includes a small number of API changes, several new features,
enhancements, and performance improvements along with a large number of bug fixes. We recommend that all
users upgrade to this version.

pandas >= 0.18.0 no longer supports compatibility with Python version 2.6
and 3.3 ([GH7718](https://github.com/pandas-dev/pandas/issues/7718), [GH11273](https://github.com/pandas-dev/pandas/issues/11273))

`numexpr` version 2.4.4 will now show a warning and not be used as a computation back-end for pandas because of some buggy behavior. This does not affect other versions (>= 2.1 and >= 2.4.6). ([GH12489](https://github.com/pandas-dev/pandas/issues/12489))

Highlights include:

- Moving and expanding window functions are now methods on Series and DataFrame,
  similar to `.groupby`, see [here](#whatsnew-0180-enhancements-moments).  
- Adding support for a `RangeIndex` as a specialized form of the `Int64Index`
  for memory savings, see [here](#whatsnew-0180-enhancements-rangeindex).  
- API breaking change to the `.resample` method to make it more `.groupby`
  like, see [here](#whatsnew-0180-breaking-resample).  
- Removal of support for positional indexing with floats, which was deprecated
  since 0.14.0. This will now raise a `TypeError`, see [here](#whatsnew-0180-float-indexers).  
- The `.to_xarray()` function has been added for compatibility with the
  [xarray package](http://xarray.pydata.org/en/stable/), see [here](#whatsnew-0180-enhancements-xarray).  
- The `read_sas` function has been enhanced to read `sas7bdat` files, see [here](#whatsnew-0180-enhancements-sas).  
- Addition of the [.str.extractall() method](#whatsnew-0180-enhancements-extract),
  and API changes to the [.str.extract() method](#whatsnew-0180-enhancements-extract)
  and [.str.cat() method](#whatsnew-0180-enhancements-strcat).  
- `pd.test()` top-level nose test runner is available ([GH4327](https://github.com/pandas-dev/pandas/issues/4327)).  


Check the [API Changes](#whatsnew-0180-api-breaking) and [deprecations](#whatsnew-0180-deprecations) before updating.

## What’s new in v0.18.0

- [New features](#New-features)  
  - [Window functions are now methods](#Window-functions-are-now-methods)  
  - [Changes to rename](#Changes-to-rename)  
  - [Range index](#Range-index)  
  - [Changes to str.extract](#Changes-to-str.extract)  
  - [Addition of str.extractall](#Addition-of-str.extractall)  
  - [Changes to str.cat](#Changes-to-str.cat)  
  - [Datetimelike rounding](#Datetimelike-rounding)  
  - [Formatting of integers in FloatIndex](#Formatting-of-integers-in-FloatIndex)  
  - [Changes to dtype assignment behaviors](#Changes-to-dtype-assignment-behaviors)  
  - [to_xarray](#to_xarray)  
  - [Latex representation](#Latex-representation)  
  - [`pd.read_sas()` changes](#`pd.read_sas()`-changes)  
  - [Other enhancements](#Other-enhancements)  
- [Backwards incompatible API changes](#Backwards-incompatible-API-changes)  
  - [NaT and Timedelta operations](#NaT-and-Timedelta-operations)  
  - [Changes to msgpack](#Changes-to-msgpack)  
  - [Signature change for .rank](#Signature-change-for-.rank)  
  - [Bug in QuarterBegin with n=0](#Bug-in-QuarterBegin-with-n=0)  
  - [Resample API](#Resample-API)  
    - [Downsampling](#Downsampling)  
    - [Upsampling](#Upsampling)  
    - [Previous API will work but with deprecations](#Previous-API-will-work-but-with-deprecations)  
  - [Changes to eval](#Changes-to-eval)  
  - [Other API changes](#Other-API-changes)  
  - [Deprecations](#Deprecations)  
  - [Removal of deprecated float indexers](#Removal-of-deprecated-float-indexers)  
  - [Removal of prior version deprecations/changes](#Removal-of-prior-version-deprecations/changes)  
- [Performance improvements](#Performance-improvements)  
- [Bug Fixes](#Bug-Fixes)  
- [Contributors](#Contributors)  


<a id='whatsnew-0180-enhancements'></a>

## New features


<a id='whatsnew-0180-enhancements-moments'></a>

### Window functions are now methods

Window functions have been refactored to be methods on `Series/DataFrame` objects, rather than top-level functions, which are now deprecated. This allows these window-type functions, to have a similar API to that of `.groupby`. See the full documentation [here](user_guide/computation.ipynb#stats-moments) ([GH11603](https://github.com/pandas-dev/pandas/issues/11603), [GH12373](https://github.com/pandas-dev/pandas/issues/12373))

Previous behavior:

```ipython
In [8]: pd.rolling_mean(df, window=3)
        FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with
                       DataFrame.rolling(window=3,center=False).mean()
Out[8]:
    A         B
0 NaN       NaN
1 NaN       NaN
2   1  0.237722
3   2 -0.023640
4   3  0.133155
5   4 -0.048693
6   5  0.342054
7   6  0.370076
8   7  0.079587
9   8 -0.954504
```


New behavior:

These show a descriptive repr

with tab-completion of available methods and properties.

```ipython
In [9]: r.<TAB>  # noqa E225, E999
r.A           r.agg         r.apply       r.count       r.exclusions  r.max         r.median      r.name        r.skew        r.sum
r.B           r.aggregate   r.corr        r.cov         r.kurt        r.mean        r.min         r.quantile    r.std         r.var
```


The methods operate on the `Rolling` object itself

They provide getitem accessors

And multiple aggregations


<a id='whatsnew-0180-enhancements-rename'></a>

### Changes to rename

`Series.rename` and `NDFrame.rename_axis` can now take a scalar or list-like
argument for altering the Series or axis *name*, in addition to their old behaviors of altering labels. ([GH9494](https://github.com/pandas-dev/pandas/issues/9494), [GH11965](https://github.com/pandas-dev/pandas/issues/11965))

The new functionality works well in method chains. Previously these methods only accepted functions or dicts mapping a *label* to a new label.
This continues to work as before for function or dict-like values.


<a id='whatsnew-0180-enhancements-rangeindex'></a>

### Range index

A `RangeIndex` has been added to the `Int64Index` sub-classes to support a memory saving alternative for common use cases. This has a similar implementation to the python `range` object (`xrange` in python 2), in that it only stores the start, stop, and step values for the index. It will transparently interact with the user API, converting to `Int64Index` if needed.

This will now be the default constructed index for `NDFrame` objects, rather than previous an `Int64Index`. ([GH939](https://github.com/pandas-dev/pandas/issues/939), [GH12070](https://github.com/pandas-dev/pandas/issues/12070), [GH12071](https://github.com/pandas-dev/pandas/issues/12071), [GH12109](https://github.com/pandas-dev/pandas/issues/12109), [GH12888](https://github.com/pandas-dev/pandas/issues/12888))

Previous behavior:

```ipython
In [3]: s = pd.Series(range(1000))

In [4]: s.index
Out[4]:
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999], dtype='int64', length=1000)

In [6]: s.index.nbytes
Out[6]: 8000
```


New behavior:


<a id='whatsnew-0180-enhancements-extract'></a>

### Changes to str.extract

The [.str.extract](user_guide/text.ipynb#text-extract) method takes a regular
expression with capture groups, finds the first match in each subject
string, and returns the contents of the capture groups
([GH11386](https://github.com/pandas-dev/pandas/issues/11386)).

In v0.18.0, the `expand` argument was added to
`extract`.

- `expand=False`: it returns a `Series`, `Index`, or `DataFrame`, depending on the subject and regular expression pattern (same behavior as pre-0.18.0).  
- `expand=True`: it always returns a `DataFrame`, which is more consistent and less confusing from the perspective of a user.  


Currently the default is `expand=None` which gives a `FutureWarning` and uses `expand=False`. To avoid this warning, please explicitly specify `expand`.

```ipython
In [1]: pd.Series(['a1', 'b2', 'c3']).str.extract(r'[ab](\d)', expand=None)
FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame)
but in a future version of pandas this will be changed to expand=True (return DataFrame)

Out[1]:
0      1
1      2
2    NaN
dtype: object
```


Extracting a regular expression with one group returns a Series if
`expand=False`.

It returns a `DataFrame` with one column if `expand=True`.

Calling on an `Index` with a regex with exactly one capture group
returns an `Index` if `expand=False`.

It returns a `DataFrame` with one column if `expand=True`.

Calling on an `Index` with a regex with more than one capture group
raises `ValueError` if `expand=False`.

In [ ]:
>>> s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=False)
ValueError: only one regex group is supported with Index

It returns a `DataFrame` if `expand=True`.

In summary, `extract(expand=True)` always returns a `DataFrame`
with a row for every subject string, and a column for every capture
group.


<a id='whatsnew-0180-enhancements-extractall'></a>

### Addition of str.extractall

The [.str.extractall](user_guide/text.ipynb#text-extractall) method was added
([GH11386](https://github.com/pandas-dev/pandas/issues/11386)).  Unlike `extract`, which returns only the first
match.

The `extractall` method returns all matches.


<a id='whatsnew-0180-enhancements-strcat'></a>

### Changes to str.cat

The method `.str.cat()` concatenates the members of a `Series`. Before, if `NaN` values were present in the Series, calling `.str.cat()` on it would return `NaN`, unlike the rest of the `Series.str.*` API. This behavior has been amended to ignore `NaN` values by default. ([GH11435](https://github.com/pandas-dev/pandas/issues/11435)).

A new, friendlier `ValueError` is added to protect against the mistake of supplying the `sep` as an arg, rather than as a kwarg. ([GH11334](https://github.com/pandas-dev/pandas/issues/11334)).

```ipython
In [2]: pd.Series(['a', 'b', np.nan, 'c']).str.cat(' ')
ValueError: Did you mean to supply a `sep` keyword?
```



<a id='whatsnew-0180-enhancements-rounding'></a>

### Datetimelike rounding

`DatetimeIndex`, `Timestamp`, `TimedeltaIndex`, `Timedelta` have gained the `.round()`, `.floor()` and `.ceil()` method for datetimelike rounding, flooring and ceiling. ([GH4314](https://github.com/pandas-dev/pandas/issues/4314), [GH11963](https://github.com/pandas-dev/pandas/issues/11963))

Naive datetimes

Tz-aware are rounded, floored and ceiled in local times

Timedeltas

In addition, `.round()`, `.floor()` and `.ceil()` will be available through the `.dt` accessor of `Series`.

### Formatting of integers in FloatIndex

Integers in `FloatIndex`, e.g. 1., are now formatted with a decimal point and a `0` digit, e.g. `1.0` ([GH11713](https://github.com/pandas-dev/pandas/issues/11713))
This change not only affects the display to the console, but also the output of IO methods like `.to_csv` or `.to_html`.

Previous behavior:

```ipython
In [2]: s = pd.Series([1, 2, 3], index=np.arange(3.))

In [3]: s
Out[3]:
0    1
1    2
2    3
dtype: int64

In [4]: s.index
Out[4]: Float64Index([0.0, 1.0, 2.0], dtype='float64')

In [5]: print(s.to_csv(path=None))
0,1
1,2
2,3
```


New behavior:

### Changes to dtype assignment behaviors

When a DataFrame’s slice is updated with a new slice of the same dtype, the dtype of the DataFrame will now remain the same. ([GH10503](https://github.com/pandas-dev/pandas/issues/10503))

Previous behavior:

```ipython
In [5]: df = pd.DataFrame({'a': [0, 1, 1],
                           'b': pd.Series([100, 200, 300], dtype='uint32')})

In [7]: df.dtypes
Out[7]:
a     int64
b    uint32
dtype: object

In [8]: ix = df['a'] == 1

In [9]: df.loc[ix, 'b'] = df.loc[ix, 'b']

In [11]: df.dtypes
Out[11]:
a    int64
b    int64
dtype: object
```


New behavior:

When a DataFrame’s integer slice is partially updated with a new slice of floats that could potentially be down-casted to integer without losing precision, the dtype of the slice will be set to float instead of integer.

Previous behavior:

```ipython
In [4]: df = pd.DataFrame(np.array(range(1,10)).reshape(3,3),
                          columns=list('abc'),
                          index=[[4,4,8], [8,10,12]])

In [5]: df
Out[5]:
      a  b  c
4 8   1  2  3
  10  4  5  6
8 12  7  8  9

In [7]: df.ix[4, 'c'] = np.array([0., 1.])

In [8]: df
Out[8]:
      a  b  c
4 8   1  2  0
  10  4  5  1
8 12  7  8  9
```


New behavior:


<a id='whatsnew-0180-enhancements-xarray'></a>

### to_xarray

In a future version of pandas, we will be deprecating `Panel` and other > 2 ndim objects. In order to provide for continuity,
all `NDFrame` objects have gained the `.to_xarray()` method in order to convert to `xarray` objects, which has
a pandas-like interface for > 2 ndim. ([GH11972](https://github.com/pandas-dev/pandas/issues/11972))

See the [xarray full-documentation here](http://xarray.pydata.org/en/stable/).

```ipython
In [1]: p = Panel(np.arange(2*3*4).reshape(2,3,4))

In [2]: p.to_xarray()
Out[2]:
<xarray.DataArray (items: 2, major_axis: 3, minor_axis: 4)>
array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])
Coordinates:
  * items       (items) int64 0 1
  * major_axis  (major_axis) int64 0 1 2
  * minor_axis  (minor_axis) int64 0 1 2 3
```


### Latex representation

`DataFrame` has gained a `._repr_latex_()` method in order to allow for conversion to latex in a ipython/jupyter notebook using nbconvert. ([GH11778](https://github.com/pandas-dev/pandas/issues/11778))

Note that this must be activated by setting the option `pd.display.latex.repr=True` ([GH12182](https://github.com/pandas-dev/pandas/issues/12182))

For example, if you have a jupyter notebook you plan to convert to latex using nbconvert, place the statement `pd.display.latex.repr=True` in the first cell to have the contained DataFrame output also stored as latex.

The options `display.latex.escape` and `display.latex.longtable` have also been added to the configuration and are used automatically by the `to_latex`
method. See the [available options docs](user_guide/options.ipynb#options-available) for more info.


<a id='whatsnew-0180-enhancements-sas'></a>

### `pd.read_sas()` changes

`read_sas` has gained the ability to read SAS7BDAT files, including compressed files.  The files can be read in entirety, or incrementally.  For full details see [here](user_guide/io.ipynb#io-sas). ([GH4052](https://github.com/pandas-dev/pandas/issues/4052))


<a id='whatsnew-0180-enhancements-other'></a>

### Other enhancements

- Handle truncated floats in SAS xport files ([GH11713](https://github.com/pandas-dev/pandas/issues/11713))  
- Added option to hide index in `Series.to_string` ([GH11729](https://github.com/pandas-dev/pandas/issues/11729))  
- `read_excel` now supports s3 urls of the format `s3://bucketname/filename` ([GH11447](https://github.com/pandas-dev/pandas/issues/11447))  
- add support for `AWS_S3_HOST` env variable when reading from s3 ([GH12198](https://github.com/pandas-dev/pandas/issues/12198))  
- A simple version of `Panel.round()` is now implemented ([GH11763](https://github.com/pandas-dev/pandas/issues/11763))  
- For Python 3.x, `round(DataFrame)`, `round(Series)`, `round(Panel)` will work ([GH11763](https://github.com/pandas-dev/pandas/issues/11763))  
- `sys.getsizeof(obj)` returns the memory usage of a pandas object, including the
  values it contains ([GH11597](https://github.com/pandas-dev/pandas/issues/11597))  
- `Series` gained an `is_unique` attribute ([GH11946](https://github.com/pandas-dev/pandas/issues/11946))  
- `DataFrame.quantile` and `Series.quantile` now accept `interpolation` keyword ([GH10174](https://github.com/pandas-dev/pandas/issues/10174)).  
- Added `DataFrame.style.format` for more flexible formatting of cell values ([GH11692](https://github.com/pandas-dev/pandas/issues/11692))  
- `DataFrame.select_dtypes` now allows the `np.float16` type code ([GH11990](https://github.com/pandas-dev/pandas/issues/11990))  
- `pivot_table()` now accepts most iterables for the `values` parameter ([GH12017](https://github.com/pandas-dev/pandas/issues/12017))  
- Added Google `BigQuery` service account authentication support, which enables authentication on remote servers. ([GH11881](https://github.com/pandas-dev/pandas/issues/11881), [GH12572](https://github.com/pandas-dev/pandas/issues/12572)). For further details see [here](https://pandas-gbq.readthedocs.io/en/latest/intro.html)  
- `HDFStore` is now iterable: `for k in store` is equivalent to `for k in store.keys()` ([GH12221](https://github.com/pandas-dev/pandas/issues/12221)).  
- Add missing methods/fields to `.dt` for `Period` ([GH8848](https://github.com/pandas-dev/pandas/issues/8848))  
- The entire code base has been `PEP`-ified ([GH12096](https://github.com/pandas-dev/pandas/issues/12096))  



<a id='whatsnew-0180-api-breaking'></a>

## Backwards incompatible API changes

- the leading white spaces have been removed from the output of `.to_string(index=False)` method ([GH11833](https://github.com/pandas-dev/pandas/issues/11833))  
- the `out` parameter has been removed from the `Series.round()` method. ([GH11763](https://github.com/pandas-dev/pandas/issues/11763))  
- `DataFrame.round()` leaves non-numeric columns unchanged in its return, rather than raises. ([GH11885](https://github.com/pandas-dev/pandas/issues/11885))  
- `DataFrame.head(0)` and `DataFrame.tail(0)` return empty frames, rather than `self`.  ([GH11937](https://github.com/pandas-dev/pandas/issues/11937))  
- `Series.head(0)` and `Series.tail(0)` return empty series, rather than `self`.  ([GH11937](https://github.com/pandas-dev/pandas/issues/11937))  
- `to_msgpack` and `read_msgpack` encoding now defaults to `'utf-8'`. ([GH12170](https://github.com/pandas-dev/pandas/issues/12170))  
- the order of keyword arguments to text file parsing functions (`.read_csv()`, `.read_table()`, `.read_fwf()`) changed to group related arguments. ([GH11555](https://github.com/pandas-dev/pandas/issues/11555))  
- `NaTType.isoformat` now returns the string `'NaT` to allow the result to
  be passed to the constructor of `Timestamp`. ([GH12300](https://github.com/pandas-dev/pandas/issues/12300))  

### NaT and Timedelta operations

`NaT` and `Timedelta` have expanded arithmetic operations, which are extended to `Series`
arithmetic where applicable.  Operations defined for `datetime64[ns]` or `timedelta64[ns]`
are now also defined for `NaT` ([GH11564](https://github.com/pandas-dev/pandas/issues/11564)).

`NaT` now supports arithmetic operations with integers and floats.

`NaT` defines more arithmetic operations with `datetime64[ns]` and `timedelta64[ns]`.

`NaT` may represent either a `datetime64[ns]` null or a `timedelta64[ns]` null.
Given the ambiguity, it is treated as a `timedelta64[ns]`, which allows more operations
to succeed.

as opposed to

```ipython
In [3]: pd.Timestamp('19900315') + pd.Timestamp('19900315')
TypeError: unsupported operand type(s) for +: 'Timestamp' and 'Timestamp'
```


However, when wrapped in a `Series` whose `dtype` is `datetime64[ns]` or `timedelta64[ns]`,
the `dtype` information is respected.

```ipython
In [1]: pd.Series([pd.NaT], dtype='<M8[ns]') + pd.Series([pd.NaT], dtype='<M8[ns]')
TypeError: can only operate on a datetimes for subtraction,
           but the operator [__add__] was passed
```


`Timedelta` division by `floats` now works.

Subtraction by `Timedelta` in a `Series` by a `Timestamp` works ([GH11925](https://github.com/pandas-dev/pandas/issues/11925))

`NaT.isoformat()` now returns `'NaT'`. This change allows allows
`pd.Timestamp` to rehydrate any timestamp like object from its isoformat
([GH12300](https://github.com/pandas-dev/pandas/issues/12300)).

### Changes to msgpack

Forward incompatible changes in `msgpack` writing format were made over 0.17.0 and 0.18.0; older versions of pandas cannot read files packed by newer versions ([GH12129](https://github.com/pandas-dev/pandas/issues/12129), [GH10527](https://github.com/pandas-dev/pandas/issues/10527))

Bugs in `to_msgpack` and `read_msgpack` introduced in 0.17.0 and fixed in 0.18.0, caused files packed in Python 2 unreadable by Python 3 ([GH12142](https://github.com/pandas-dev/pandas/issues/12142)). The following table describes the backward and forward compat of msgpacks.

-   
-   


|Packed with|Can be unpacked with|
|:--------------------:|:----------------------:|
|pre-0.17 / Python 2|any|
|pre-0.17 / Python 3|any|
|0.17 / Python 2|==0.17 / Python 2>=0.18 / any Python|
|0.17 / Python 3|>=0.18 / any Python|
|0.18|>= 0.18|
0.18.0 is backward-compatible for reading files packed by older versions, except for files packed with 0.17 in Python 2, in which case only they can only be unpacked in Python 2.

### Signature change for .rank

`Series.rank` and `DataFrame.rank` now have the same signature ([GH11759](https://github.com/pandas-dev/pandas/issues/11759))

Previous signature

```ipython
In [3]: pd.Series([0,1]).rank(method='average', na_option='keep',
                              ascending=True, pct=False)
Out[3]:
0    1
1    2
dtype: float64

In [4]: pd.DataFrame([0,1]).rank(axis=0, numeric_only=None,
                                 method='average', na_option='keep',
                                 ascending=True, pct=False)
Out[4]:
   0
0  1
1  2
```


New signature

### Bug in QuarterBegin with n=0

In previous versions, the behavior of the QuarterBegin offset was inconsistent
depending on the date when the `n` parameter was 0. ([GH11406](https://github.com/pandas-dev/pandas/issues/11406))

The general semantics of anchored offsets for `n=0` is to not move the date
when it is an anchor point (e.g., a quarter start date), and otherwise roll
forward to the next anchor point.

For the `QuarterBegin` offset in previous versions, the date would be rolled
*backwards* if date was in the same month as the quarter start date.

```ipython
In [3]: d = pd.Timestamp('2014-02-15')

In [4]: d + pd.offsets.QuarterBegin(n=0, startingMonth=2)
Out[4]: Timestamp('2014-02-01 00:00:00')
```


This behavior has been corrected in version 0.18.0, which is consistent with
other anchored offsets like `MonthBegin` and `YearBegin`.


<a id='whatsnew-0180-breaking-resample'></a>

### Resample API

Like the change in the window functions API [above](#whatsnew-0180-enhancements-moments), `.resample(...)` is changing to have a more groupby-like API. ([GH11732](https://github.com/pandas-dev/pandas/issues/11732), [GH12702](https://github.com/pandas-dev/pandas/issues/12702), [GH12202](https://github.com/pandas-dev/pandas/issues/12202), [GH12332](https://github.com/pandas-dev/pandas/issues/12332), [GH12334](https://github.com/pandas-dev/pandas/issues/12334), [GH12348](https://github.com/pandas-dev/pandas/issues/12348), [GH12448](https://github.com/pandas-dev/pandas/issues/12448)).

**Previous API**:

You would write a resampling operation that immediately evaluates. If a `how` parameter was not provided, it
would default to `how='mean'`.

```ipython
In [6]: df.resample('2s')
Out[6]:
                         A         B         C         D
2010-01-01 09:00:00  0.485748  0.447351  0.357096  0.793615
2010-01-01 09:00:02  0.820801  0.794317  0.364034  0.531096
2010-01-01 09:00:04  0.433985  0.314582  0.424104  0.625733
2010-01-01 09:00:06  0.624988  0.609738  0.633165  0.612452
2010-01-01 09:00:08  0.510470  0.534317  0.573201  0.806949
```


You could also specify a `how` directly

```ipython
In [7]: df.resample('2s', how='sum')
Out[7]:
                         A         B         C         D
2010-01-01 09:00:00  0.971495  0.894701  0.714192  1.587231
2010-01-01 09:00:02  1.641602  1.588635  0.728068  1.062191
2010-01-01 09:00:04  0.867969  0.629165  0.848208  1.251465
2010-01-01 09:00:06  1.249976  1.219477  1.266330  1.224904
2010-01-01 09:00:08  1.020940  1.068634  1.146402  1.613897
```


**New API**:

Now, you can write `.resample(..)` as a 2-stage operation like `.groupby(...)`, which
yields a `Resampler`.

#### Downsampling

You can then use this object to perform operations.
These are downsampling operations (going from a higher frequency to a lower one).

Furthermore, resample now supports `getitem` operations to perform the resample on specific columns.

and `.aggregate` type operations.

These accessors can of course, be combined

#### Upsampling

Upsampling operations take you from a lower frequency to a higher frequency. These are now
performed with the `Resampler` objects with `backfill()`,
`ffill()`, `fillna()` and `asfreq()` methods.

Previously

```ipython
In [6]: s.resample('M', fill_method='ffill')
Out[6]:
2010-03-31    0
2010-04-30    0
2010-05-31    0
2010-06-30    1
2010-07-31    1
2010-08-31    1
2010-09-30    2
2010-10-31    2
2010-11-30    2
2010-12-31    3
2011-01-31    3
2011-02-28    3
2011-03-31    4
Freq: M, dtype: int64
```


New API

>**Note**
>
>In the new API, you can either downsample OR upsample. The prior implementation would allow you to pass an aggregator function (like `mean`) even though you were upsampling, providing a bit of confusion.

#### Previous API will work but with deprecations

This new API for resample includes some internal changes for the prior-to-0.18.0 API, to work with a deprecation warning in most cases, as the resample operation returns a deferred object. We can intercept operations and just do what the (pre 0.18.0) API did (with a warning). Here is a typical use case:

```ipython
In [4]: r = df.resample('2s')

In [6]: r*10
pandas/tseries/resample.py:80: FutureWarning: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)

Out[6]:
                      A         B         C         D
2010-01-01 09:00:00  4.857476  4.473507  3.570960  7.936154
2010-01-01 09:00:02  8.208011  7.943173  3.640340  5.310957
2010-01-01 09:00:04  4.339846  3.145823  4.241039  6.257326
2010-01-01 09:00:06  6.249881  6.097384  6.331650  6.124518
2010-01-01 09:00:08  5.104699  5.343172  5.732009  8.069486
```


However, getting and assignment operations directly on a `Resampler` will raise a `ValueError`:

```ipython
In [7]: r.iloc[0] = 5
ValueError: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)
```


There is a situation where the new API can not perform all the operations when using original code.
This code is intending to resample every 2s, take the `mean` AND then take the `min` of those results.

```ipython
In [4]: df.resample('2s').min()
Out[4]:
A    0.433985
B    0.314582
C    0.357096
D    0.531096
dtype: float64
```


The new API will:

The good news is the return dimensions will differ between the new API and the old API, so this should loudly raise
an exception.

To replicate the original operation

### Changes to eval

In prior versions, new columns assignments in an `eval` expression resulted
in an inplace change to the `DataFrame`. ([GH9297](https://github.com/pandas-dev/pandas/issues/9297), [GH8664](https://github.com/pandas-dev/pandas/issues/8664), [GH10486](https://github.com/pandas-dev/pandas/issues/10486))

```ipython
In [12]: df.eval('c = a + b')
FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.

In [13]: df
Out[13]:
      a  b     c
0   0.0  0   0.0
1   2.5  1   3.5
2   5.0  2   7.0
3   7.5  3  10.5
4  10.0  4  14.0
```


In version 0.18.0, a new `inplace` keyword was added to choose whether the
assignment should be done inplace or return a copy.

For backwards compatibility, `inplace` defaults to `True` if not specified.
This will change in a future version of pandas. If your code depends on an
inplace assignment you should update to explicitly set `inplace=True`

The `inplace` keyword parameter was also added the `query` method.

Note that the default value for `inplace` in a `query`
is `False`, which is consistent with prior versions.

`eval` has also been updated to allow multi-line expressions for multiple
assignments.  These expressions will be evaluated one at a time in order.  Only
assignments are valid for multi-line expressions.


<a id='whatsnew-0180-api'></a>

### Other API changes

- `DataFrame.between_time` and `Series.between_time` now only parse a fixed set of time strings. Parsing of date strings is no longer supported and raises a `ValueError`. ([GH11818](https://github.com/pandas-dev/pandas/issues/11818))  
This will now raise.  

```ipython
In [2]: s.between_time('20150101 07:00:00','20150101 09:00:00')
ValueError: Cannot convert arg ['20150101 07:00:00'] to a time.
```


- `.memory_usage()` now includes values in the index, as does memory_usage in `.info()` ([GH11597](https://github.com/pandas-dev/pandas/issues/11597))  
- `DataFrame.to_latex()` now supports non-ascii encodings (eg `utf-8`) in Python 2 with the parameter `encoding` ([GH7061](https://github.com/pandas-dev/pandas/issues/7061))  
- `pandas.merge()` and `DataFrame.merge()` will show a specific error message when trying to merge with an object that is not of type `DataFrame` or a subclass ([GH12081](https://github.com/pandas-dev/pandas/issues/12081))  
- `DataFrame.unstack` and `Series.unstack` now take `fill_value` keyword to allow direct replacement of missing values when an unstack results in missing values in the resulting `DataFrame`. As an added benefit, specifying `fill_value` will preserve the data type of the original stacked data.  ([GH9746](https://github.com/pandas-dev/pandas/issues/9746))  
- As part of the new API for [window functions](#whatsnew-0180-enhancements-moments) and [resampling](#whatsnew-0180-breaking-resample), aggregation functions have been clarified, raising more informative error messages on invalid aggregations. ([GH9052](https://github.com/pandas-dev/pandas/issues/9052)). A full set of examples are presented in [groupby](user_guide/groupby.ipynb#groupby-aggregate).  
- Statistical functions for `NDFrame` objects (like `sum(), mean(), min()`) will now raise if non-numpy-compatible arguments are passed in for `**kwargs` ([GH12301](https://github.com/pandas-dev/pandas/issues/12301))  
- `.to_latex` and `.to_html` gain a `decimal` parameter like `.to_csv`; the default is `'.'` ([GH12031](https://github.com/pandas-dev/pandas/issues/12031))  
- More helpful error message when constructing a `DataFrame` with empty data but with indices ([GH8020](https://github.com/pandas-dev/pandas/issues/8020))  
- `.describe()` will now properly handle bool dtype as a categorical ([GH6625](https://github.com/pandas-dev/pandas/issues/6625))  
- More helpful error message with an invalid `.transform` with user defined input ([GH10165](https://github.com/pandas-dev/pandas/issues/10165))  
- Exponentially weighted functions now allow specifying alpha directly ([GH10789](https://github.com/pandas-dev/pandas/issues/10789)) and raise `ValueError` if parameters violate `0 < alpha <= 1` ([GH12492](https://github.com/pandas-dev/pandas/issues/12492))  



<a id='whatsnew-0180-deprecations'></a>

### Deprecations


<a id='whatsnew-0180-window-deprecations'></a>
- The functions `pd.rolling_*`, `pd.expanding_*`, and `pd.ewm*` are deprecated and replaced by the corresponding method call. Note that
the new suggested syntax includes all of the arguments (even if default) ([GH11603](https://github.com/pandas-dev/pandas/issues/11603))  

```ipython
In [1]: s = pd.Series(range(3))

In [2]: pd.rolling_mean(s,window=2,min_periods=1)
        FutureWarning: pd.rolling_mean is deprecated for Series and
             will be removed in a future version, replace with
             Series.rolling(min_periods=1,window=2,center=False).mean()
Out[2]:
        0    0.0
        1    0.5
        2    1.5
        dtype: float64

In [3]: pd.rolling_cov(s, s, window=2)
        FutureWarning: pd.rolling_cov is deprecated for Series and
             will be removed in a future version, replace with
             Series.rolling(window=2).cov(other=<Series>)
Out[3]:
        0    NaN
        1    0.5
        2    0.5
        dtype: float64
```


- The `freq` and `how` arguments to the `.rolling`, `.expanding`, and `.ewm` (new) functions are deprecated, and will be removed in a future version. You can simply resample the input prior to creating a window function. ([GH11603](https://github.com/pandas-dev/pandas/issues/11603)).  
  For example, instead of `s.rolling(window=5,freq='D').max()` to get the max value on a rolling 5 Day window, one could use `s.resample('D').mean().rolling(window=5).max()`, which first resamples the data to daily data, then provides a rolling 5 day window.  
- `pd.tseries.frequencies.get_offset_name` function is deprecated. Use offset’s `.freqstr` property as alternative ([GH11192](https://github.com/pandas-dev/pandas/issues/11192))  
- `pandas.stats.fama_macbeth` routines are deprecated and will be removed in a future version ([GH6077](https://github.com/pandas-dev/pandas/issues/6077))  
- `pandas.stats.ols`, `pandas.stats.plm` and `pandas.stats.var` routines are deprecated and will be removed in a future version ([GH6077](https://github.com/pandas-dev/pandas/issues/6077))  
- show a `FutureWarning` rather than a `DeprecationWarning` on using long-time deprecated syntax in `HDFStore.select`, where the `where` clause is not a string-like ([GH12027](https://github.com/pandas-dev/pandas/issues/12027))  
- The `pandas.options.display.mpl_style` configuration has been deprecated
  and will be removed in a future version of pandas. This functionality
  is better handled by matplotlib’s [style sheets](http://matplotlib.org/users/style_sheets.html) ([GH11783](https://github.com/pandas-dev/pandas/issues/11783)).  



<a id='whatsnew-0180-float-indexers'></a>

### Removal of deprecated float indexers

In [GH4892](https://github.com/pandas-dev/pandas/issues/4892) indexing with floating point numbers on a non-`Float64Index` was deprecated (in version 0.14.0).
In 0.18.0, this deprecation warning is removed and these will now raise a `TypeError`. ([GH12165](https://github.com/pandas-dev/pandas/issues/12165), [GH12333](https://github.com/pandas-dev/pandas/issues/12333))

Previous behavior:

```ipython
# this is label indexing
In [2]: s[5.0]
FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
Out[2]: 2

# this is positional indexing
In [3]: s.iloc[1.0]
FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
Out[3]: 2

# this is label indexing
In [4]: s.loc[5.0]
FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
Out[4]: 2

# .ix would coerce 1.0 to the positional 1, and index
In [5]: s2.ix[1.0] = 10
FutureWarning: scalar indexers for index type Index should be integers and not floating point

In [6]: s2
Out[6]:
a     1
b    10
c     3
dtype: int64
```


New behavior:

For iloc, getting & setting via a float scalar will always raise.

```ipython
In [3]: s.iloc[2.0]
TypeError: cannot do label indexing on <class 'pandas.indexes.numeric.Int64Index'> with these indexers [2.0] of <type 'float'>
```


Other indexers will coerce to a like integer for both getting and setting. The `FutureWarning` has been dropped for `.loc`, `.ix` and `[]`.

and setting

Positional setting with `.ix` and a float indexer will ADD this value to the index, rather than previously setting the value by position.

```ipython
In [3]: s2.ix[1.0] = 10
In [4]: s2
Out[4]:
a       1
b       2
c       3
1.0    10
dtype: int64
```


Slicing will also coerce integer-like floats to integers for a non-`Float64Index`.

Note that for floats that are NOT coercible to ints, the label based bounds will be excluded

Float indexing on a `Float64Index` is unchanged.


<a id='whatsnew-0180-prior-deprecations'></a>

### Removal of prior version deprecations/changes

- Removal of `rolling_corr_pairwise` in favor of `.rolling().corr(pairwise=True)` ([GH4950](https://github.com/pandas-dev/pandas/issues/4950))  
- Removal of `expanding_corr_pairwise` in favor of `.expanding().corr(pairwise=True)` ([GH4950](https://github.com/pandas-dev/pandas/issues/4950))  
- Removal of `DataMatrix` module. This was not imported into the pandas namespace in any event ([GH12111](https://github.com/pandas-dev/pandas/issues/12111))  
- Removal of `cols` keyword in favor of `subset` in `DataFrame.duplicated()` and `DataFrame.drop_duplicates()` ([GH6680](https://github.com/pandas-dev/pandas/issues/6680))  
- Removal of the `read_frame` and `frame_query` (both aliases for `pd.read_sql`)
  and `write_frame` (alias of `to_sql`) functions in the `pd.io.sql` namespace,
  deprecated since 0.14.0 ([GH6292](https://github.com/pandas-dev/pandas/issues/6292)).  
- Removal of the `order` keyword from `.factorize()` ([GH6930](https://github.com/pandas-dev/pandas/issues/6930))  



<a id='whatsnew-0180-performance'></a>

## Performance improvements

- Improved performance of `andrews_curves` ([GH11534](https://github.com/pandas-dev/pandas/issues/11534))  
- Improved huge `DatetimeIndex`, `PeriodIndex` and `TimedeltaIndex`’s ops performance including `NaT` ([GH10277](https://github.com/pandas-dev/pandas/issues/10277))  
- Improved performance of `pandas.concat` ([GH11958](https://github.com/pandas-dev/pandas/issues/11958))  
- Improved performance of `StataReader` ([GH11591](https://github.com/pandas-dev/pandas/issues/11591))  
- Improved performance in construction of `Categoricals` with `Series` of datetimes containing `NaT` ([GH12077](https://github.com/pandas-dev/pandas/issues/12077))  
- Improved performance of ISO 8601 date parsing for dates without separators ([GH11899](https://github.com/pandas-dev/pandas/issues/11899)), leading zeros ([GH11871](https://github.com/pandas-dev/pandas/issues/11871)) and with white space preceding the time zone ([GH9714](https://github.com/pandas-dev/pandas/issues/9714))  



<a id='whatsnew-0180-bug-fixes'></a>

## Bug Fixes

- Bug in `GroupBy.size` when data-frame is empty. ([GH11699](https://github.com/pandas-dev/pandas/issues/11699))  
- Bug in `Period.end_time` when a multiple of time period is requested ([GH11738](https://github.com/pandas-dev/pandas/issues/11738))  
- Regression in `.clip` with tz-aware datetimes ([GH11838](https://github.com/pandas-dev/pandas/issues/11838))  
- Bug in `date_range` when the boundaries fell on the frequency ([GH11804](https://github.com/pandas-dev/pandas/issues/11804), [GH12409](https://github.com/pandas-dev/pandas/issues/12409))  
- Bug in consistency of passing nested dicts to `.groupby(...).agg(...)` ([GH9052](https://github.com/pandas-dev/pandas/issues/9052))  
- Accept unicode in `Timedelta` constructor ([GH11995](https://github.com/pandas-dev/pandas/issues/11995))  
- Bug in value label reading for `StataReader` when reading incrementally ([GH12014](https://github.com/pandas-dev/pandas/issues/12014))  
- Bug in vectorized `DateOffset` when `n` parameter is `0` ([GH11370](https://github.com/pandas-dev/pandas/issues/11370))  
- Compat for numpy 1.11 w.r.t. `NaT` comparison changes ([GH12049](https://github.com/pandas-dev/pandas/issues/12049))  
- Bug in `read_csv` when reading from a `StringIO` in threads ([GH11790](https://github.com/pandas-dev/pandas/issues/11790))  
- Bug in not treating `NaT` as a missing value in datetimelikes when factorizing & with `Categoricals` ([GH12077](https://github.com/pandas-dev/pandas/issues/12077))  
- Bug in getitem when the values of a `Series` were tz-aware ([GH12089](https://github.com/pandas-dev/pandas/issues/12089))  
- Bug in `Series.str.get_dummies` when one of the variables was ‘name’ ([GH12180](https://github.com/pandas-dev/pandas/issues/12180))  
- Bug in `pd.concat` while concatenating tz-aware NaT series. ([GH11693](https://github.com/pandas-dev/pandas/issues/11693), [GH11755](https://github.com/pandas-dev/pandas/issues/11755), [GH12217](https://github.com/pandas-dev/pandas/issues/12217))  
- Bug in `pd.read_stata` with version <= 108 files ([GH12232](https://github.com/pandas-dev/pandas/issues/12232))  
- Bug in `Series.resample` using a frequency of `Nano` when the index is a `DatetimeIndex` and contains non-zero nanosecond parts ([GH12037](https://github.com/pandas-dev/pandas/issues/12037))  
- Bug in resampling with `.nunique` and a sparse index ([GH12352](https://github.com/pandas-dev/pandas/issues/12352))  
- Removed some compiler warnings ([GH12471](https://github.com/pandas-dev/pandas/issues/12471))  
- Work around compat issues with `boto` in python 3.5 ([GH11915](https://github.com/pandas-dev/pandas/issues/11915))  
- Bug in `NaT` subtraction from `Timestamp` or `DatetimeIndex` with timezones ([GH11718](https://github.com/pandas-dev/pandas/issues/11718))  
- Bug in subtraction of `Series` of a single tz-aware `Timestamp` ([GH12290](https://github.com/pandas-dev/pandas/issues/12290))  
- Use compat iterators in PY2 to support `.next()` ([GH12299](https://github.com/pandas-dev/pandas/issues/12299))  
- Bug in `Timedelta.round` with negative values ([GH11690](https://github.com/pandas-dev/pandas/issues/11690))  
- Bug in `.loc` against `CategoricalIndex` may result in normal `Index` ([GH11586](https://github.com/pandas-dev/pandas/issues/11586))  
- Bug in `DataFrame.info` when duplicated column names exist ([GH11761](https://github.com/pandas-dev/pandas/issues/11761))  
- Bug in `.copy` of datetime tz-aware objects ([GH11794](https://github.com/pandas-dev/pandas/issues/11794))  
- Bug in `Series.apply` and `Series.map` where `timedelta64` was not boxed ([GH11349](https://github.com/pandas-dev/pandas/issues/11349))  
- Bug in `DataFrame.set_index()` with tz-aware `Series` ([GH12358](https://github.com/pandas-dev/pandas/issues/12358))  
- Bug in subclasses of `DataFrame` where `AttributeError` did not propagate ([GH11808](https://github.com/pandas-dev/pandas/issues/11808))  
- Bug groupby on tz-aware data where selection not returning `Timestamp` ([GH11616](https://github.com/pandas-dev/pandas/issues/11616))  
- Bug in `pd.read_clipboard` and `pd.to_clipboard` functions not supporting Unicode; upgrade included `pyperclip` to v1.5.15 ([GH9263](https://github.com/pandas-dev/pandas/issues/9263))  
- Bug in `DataFrame.query` containing an assignment ([GH8664](https://github.com/pandas-dev/pandas/issues/8664))  
- Bug in `from_msgpack` where `__contains__()` fails for columns of the unpacked `DataFrame`, if the `DataFrame` has object columns. ([GH11880](https://github.com/pandas-dev/pandas/issues/11880))  
- Bug in `.resample` on categorical data with `TimedeltaIndex` ([GH12169](https://github.com/pandas-dev/pandas/issues/12169))  
- Bug in timezone info lost when broadcasting scalar datetime to `DataFrame` ([GH11682](https://github.com/pandas-dev/pandas/issues/11682))  
- Bug in `Index` creation from `Timestamp` with mixed tz coerces to UTC ([GH11488](https://github.com/pandas-dev/pandas/issues/11488))  
- Bug in `to_numeric` where it does not raise if input is more than one dimension ([GH11776](https://github.com/pandas-dev/pandas/issues/11776))  
- Bug in parsing timezone offset strings with non-zero minutes ([GH11708](https://github.com/pandas-dev/pandas/issues/11708))  
- Bug in `df.plot` using incorrect colors for bar plots under matplotlib 1.5+ ([GH11614](https://github.com/pandas-dev/pandas/issues/11614))  
- Bug in the `groupby` `plot` method when using keyword arguments ([GH11805](https://github.com/pandas-dev/pandas/issues/11805)).  
- Bug in `DataFrame.duplicated` and `drop_duplicates` causing spurious matches when setting `keep=False` ([GH11864](https://github.com/pandas-dev/pandas/issues/11864))  
- Bug in `.loc` result with duplicated key may have `Index` with incorrect dtype ([GH11497](https://github.com/pandas-dev/pandas/issues/11497))  
- Bug in `pd.rolling_median` where memory allocation failed even with sufficient memory ([GH11696](https://github.com/pandas-dev/pandas/issues/11696))  
- Bug in `DataFrame.style` with spurious zeros ([GH12134](https://github.com/pandas-dev/pandas/issues/12134))  
- Bug in `DataFrame.style` with integer columns not starting at 0 ([GH12125](https://github.com/pandas-dev/pandas/issues/12125))  
- Bug in `.style.bar` may not rendered properly using specific browser ([GH11678](https://github.com/pandas-dev/pandas/issues/11678))  
- Bug in rich comparison of `Timedelta` with a `numpy.array` of `Timedelta` that caused an infinite recursion ([GH11835](https://github.com/pandas-dev/pandas/issues/11835))  
- Bug in `DataFrame.round` dropping column index name ([GH11986](https://github.com/pandas-dev/pandas/issues/11986))  
- Bug in `df.replace` while replacing value in mixed dtype `Dataframe` ([GH11698](https://github.com/pandas-dev/pandas/issues/11698))  
- Bug in `Index` prevents copying name of passed `Index`, when a new name is not provided ([GH11193](https://github.com/pandas-dev/pandas/issues/11193))  
- Bug in `read_excel` failing to read any non-empty sheets when empty sheets exist and `sheetname=None` ([GH11711](https://github.com/pandas-dev/pandas/issues/11711))  
- Bug in `read_excel` failing to raise `NotImplemented` error when keywords `parse_dates` and `date_parser` are provided ([GH11544](https://github.com/pandas-dev/pandas/issues/11544))  
- Bug in `read_sql` with `pymysql` connections failing to return chunked data ([GH11522](https://github.com/pandas-dev/pandas/issues/11522))  
- Bug in `.to_csv` ignoring formatting parameters `decimal`, `na_rep`, `float_format` for float indexes ([GH11553](https://github.com/pandas-dev/pandas/issues/11553))  
- Bug in `Int64Index` and `Float64Index` preventing the use of the modulo operator ([GH9244](https://github.com/pandas-dev/pandas/issues/9244))  
- Bug in `MultiIndex.drop` for not lexsorted MultiIndexes ([GH12078](https://github.com/pandas-dev/pandas/issues/12078))  
- Bug in `DataFrame` when masking an empty `DataFrame` ([GH11859](https://github.com/pandas-dev/pandas/issues/11859))  
- Bug in `.plot` potentially modifying the `colors` input when the number of columns didn’t match the number of series provided ([GH12039](https://github.com/pandas-dev/pandas/issues/12039)).  
- Bug in `Series.plot` failing when index has a `CustomBusinessDay` frequency ([GH7222](https://github.com/pandas-dev/pandas/issues/7222)).  
- Bug in `.to_sql` for `datetime.time` values with sqlite fallback ([GH8341](https://github.com/pandas-dev/pandas/issues/8341))  
- Bug in `read_excel` failing to read data with one column when `squeeze=True` ([GH12157](https://github.com/pandas-dev/pandas/issues/12157))  
- Bug in `read_excel` failing to read one empty column ([GH12292](https://github.com/pandas-dev/pandas/issues/12292), [GH9002](https://github.com/pandas-dev/pandas/issues/9002))  
- Bug in `.groupby` where a `KeyError` was not raised for a wrong column if there was only one row in the dataframe ([GH11741](https://github.com/pandas-dev/pandas/issues/11741))  
- Bug in `.read_csv` with dtype specified on empty data producing an error ([GH12048](https://github.com/pandas-dev/pandas/issues/12048))  
- Bug in `.read_csv` where strings like `'2E'` are treated as valid floats ([GH12237](https://github.com/pandas-dev/pandas/issues/12237))  
- Bug in building *pandas* with debugging symbols ([GH12123](https://github.com/pandas-dev/pandas/issues/12123))  
- Removed `millisecond` property of `DatetimeIndex`. This would always raise a `ValueError` ([GH12019](https://github.com/pandas-dev/pandas/issues/12019)).  
- Bug in `Series` constructor with read-only data ([GH11502](https://github.com/pandas-dev/pandas/issues/11502))  
- Removed `pandas.util.testing.choice()`.  Should use `np.random.choice()`, instead. ([GH12386](https://github.com/pandas-dev/pandas/issues/12386))  
- Bug in `.loc` setitem indexer preventing the use of a TZ-aware DatetimeIndex ([GH12050](https://github.com/pandas-dev/pandas/issues/12050))  
- Bug in `.style` indexes and MultiIndexes not appearing ([GH11655](https://github.com/pandas-dev/pandas/issues/11655))  
- Bug in `to_msgpack` and `from_msgpack` which did not correctly serialize or deserialize `NaT` ([GH12307](https://github.com/pandas-dev/pandas/issues/12307)).  
- Bug in `.skew` and `.kurt` due to roundoff error for highly similar values ([GH11974](https://github.com/pandas-dev/pandas/issues/11974))  
- Bug in `Timestamp` constructor where microsecond resolution was lost if HHMMSS were not separated with ‘:’ ([GH10041](https://github.com/pandas-dev/pandas/issues/10041))  
- Bug in `buffer_rd_bytes` src->buffer could be freed more than once if reading failed, causing a segfault ([GH12098](https://github.com/pandas-dev/pandas/issues/12098))  
- Bug in `crosstab` where arguments with non-overlapping indexes would return a `KeyError` ([GH10291](https://github.com/pandas-dev/pandas/issues/10291))  
- Bug in `DataFrame.apply` in which reduction was not being prevented for cases in which `dtype` was not a numpy dtype ([GH12244](https://github.com/pandas-dev/pandas/issues/12244))  
- Bug when initializing categorical series with a scalar value. ([GH12336](https://github.com/pandas-dev/pandas/issues/12336))  
- Bug when specifying a UTC `DatetimeIndex` by setting `utc=True` in `.to_datetime` ([GH11934](https://github.com/pandas-dev/pandas/issues/11934))  
- Bug when increasing the buffer size of CSV reader in `read_csv` ([GH12494](https://github.com/pandas-dev/pandas/issues/12494))  
- Bug when setting columns of a `DataFrame` with duplicate column names ([GH12344](https://github.com/pandas-dev/pandas/issues/12344))  



<a id='whatsnew-0-18-0-contributors'></a>

## Contributors

A total of 101 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- ARF +  
- Alex Alekseyev +  
- Andrew McPherson +  
- Andrew Rosenfeld  
- Andy Hayden  
- Anthonios Partheniou  
- Anton I. Sipos  
- Ben +  
- Ben North +  
- Bran Yang +  
- Chris  
- Chris Carroux +  
- Christopher C. Aycock +  
- Christopher Scanlin +  
- Cody +  
- Da Wang +  
- Daniel Grady +  
- Dorozhko Anton +  
- Dr-Irv +  
- Erik M. Bray +  
- Evan Wright  
- Francis T. O’Donovan +  
- Frank Cleary +  
- Gianluca Rossi  
- Graham Jeffries +  
- Guillaume Horel  
- Henry Hammond +  
- Isaac Schwabacher +  
- Jean-Mathieu Deschenes  
- Jeff Reback  
- Joe Jevnik +  
- John Freeman +  
- John Fremlin +  
- Jonas Hoersch +  
- Joris Van den Bossche  
- Joris Vankerschaver  
- Justin Lecher  
- Justin Lin +  
- Ka Wo Chen  
- Keming Zhang +  
- Kerby Shedden  
- Kyle +  
- Marco Farrugia +  
- MasonGallo +  
- MattRijk +  
- Matthew Lurie +  
- Maximilian Roos  
- Mayank Asthana +  
- Mortada Mehyar  
- Moussa Taifi +  
- Navreet Gill +  
- Nicolas Bonnotte  
- Paul Reiners +  
- Philip Gura +  
- Pietro Battiston  
- RahulHP +  
- Randy Carnevale  
- Rinoc Johnson  
- Rishipuri +  
- Sangmin Park +  
- Scott E Lasley  
- Sereger13 +  
- Shannon Wang +  
- Skipper Seabold  
- Thierry Moisan  
- Thomas A Caswell  
- Toby Dylan Hocking +  
- Tom Augspurger  
- Travis +  
- Trent Hauck  
- Tux1  
- Varun  
- Wes McKinney  
- Will Thompson +  
- Yoav Ram  
- Yoong Kang Lim +  
- Yoshiki Vázquez Baeza  
- Young Joong Kim +  
- Younggun Kim  
- Yuval Langer +  
- alex argunov +  
- behzad nouri  
- boombard +  
- brian-pantano +  
- chromy +  
- daniel +  
- dgram0 +  
- gfyoung +  
- hack-c +  
- hcontrast +  
- jfoo +  
- kaustuv deolal +  
- llllllllll  
- ranarag +  
- rockg  
- scls19fr  
- seales +  
- sinhrks  
- srib +  
- surveymedia.ca +  
- tworec +  